# Training analysis for DeepRacer

This notebook has been built based on the `DeepRacer Log Analysis.ipynb` provided by the AWS DeepRacer Team. It has been reorganised and expanded to provide new views on the training data without the helper code which was moved into utility `.py` files.

## Usage

I have expanded this notebook from to present how I'm using this information. It contains descriptions that you may find not that needed after initial reading. Since this file can change in the future, I recommend that you make its copy and reorganize it to your liking. This way you will not lose your changes and you'll be able to add things as you please.

**This notebook isn't complete.** What I find interesting in the logs may not be what you will find interesting and useful. I recommend you get familiar with the tools and try hacking around to get the insights that suit your needs.

## Contributions

As usual, your ideas are very welcome and encouraged so if you have any suggestions either bring them to [the AWS DeepRacer Community](http://join.deepracing.io) or share as code contributions.

## Training environments

Depending on whether you're running your training through the console or using the local setup, and on which setup for local training you're using, your experience will vary. As much as I would like everything to be taylored to your configuration, there may be some problems that you may face. If so, please get in touch through [the AWS DeepRacer Community](http://join.deepracing.io).

## Requirements

Before you start using the notebook, you will need to install some dependencies. If you haven't yet done so, have a look at [The README.md file](/edit/README.md#running-the-notebooks) to find what you need to install.

Apart from the install, you also have to configure your programmatic access to AWS. Have a look at the guides below, AWS resources will lead you by the hand:

AWS CLI: https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html

Boto Configuration: https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html

## Credits

I would like to thank [the AWS DeepRacer Community](http://join.deepracing.io) for all the feedback about the notebooks. If you'd like, follow [my blog](https://codelikeamother.uk) where I tend to write about my experiences with AWS DeepRacer.

# Log Analysis

Let's get to it.

## Permissions

Depending on where you are downloading the data from, you will need some permissions:
* Access to CloudWatch log streams
* Access to S3 bucket to reach the log files

## Installs and setups

If you are using an AWS SageMaker Notebook to run the log analysis, you will need to ensure you install required dependencies. To do that uncomment and run the following:

In [1]:
!pwd

/workspace


In [2]:
# Make sure you have deepracer-utils >= 0.9

# import sys

# !{sys.executable} -m pip install --upgrade deepracer-utils

## Imports

Run the imports block below:

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

from deepracer.tracks import TrackIO, Track
from deepracer.tracks.track_utils import track_breakdown, track_meta
from deepracer.logs import \
    SimulationLogsIO as slio, \
    NewRewardUtils as nr, \
    AnalysisUtils as au, \
    PlottingUtils as pu, \
    ActionBreakdownUtils as abu, \
    DeepRacerLog

# Ignore deprecation warnings we have no power over
import warnings
warnings.filterwarnings('ignore')

## Get the logs

Depending on which way you are training your model, you will need a slightly different way to load the data. 

**AWS DeepRacer Console**

The logs can be downloaded from the training page. Once you download them, extract the archive into logs/[training-name] (just like logs/sample-logs)

**DeepRacer for Cloud**
    
If you're using local training, just point at your model's root folder in the minio bucket. If you're using any of the cloudy deployments, download the model folder to local and point at it.

**Deepracer for dummies/Chris Rhodes' Deepracer/ARCC Deepracer or any training solution other than the ones above, read below**

This notebook has been updated to support the most recent setups. Most of the mentioned projects above are no longer compatible with AWS DeepRacer Console anyway so do consider moving to the ones actively maintained.
    

In [15]:
import pandas as pd
from deepracer.logs import \
    SimulationLogsIO as slio, \
    NewRewardUtils as nr, \
    AnalysisUtils as au, \
    PlottingUtils as pu, \
    ActionBreakdownUtils as abu, \
    DeepRacerLog

In [16]:
from deepracer.logs import (AnalysisUtils, DeepRacerLog, S3FileHandler)

BUCKET="bucket"
PREFIX="rl-deepracer-101"
# WARNING the IP Address for the 's3_endpoint_url' is the IP adress of the MinIO container. THIS CAN BE THE IP WITHIN A DOCKER NETWORK
fh = S3FileHandler(bucket=BUCKET, prefix=PREFIX,
                   profile="minio", s3_endpoint_url="http://minio:9000")

log = DeepRacerLog(filehandler=fh)


In [17]:
# load logs into a dataframe
log.load()

In [18]:
hyperparameters = None
try:
    print("AGENT AND NETWORK")
    pprint(log.agent_and_network())
except Exception:
        print("Robomaker logs not available")
try:
    print("-------------")
    print("HYPERPARAMETERS")
    hyperparameters = log.hyperparameters()
    pprint(log.hyperparameters())
except Exception:
        print("Hyperparameters not available")
    
try:
    print("-------------")
    print("ACTION SPACE")
    pprint(log.action_space())
except Exception:
        print("Action space not available")
df = log.dataframe()

simulation_agg = AnalysisUtils.simulation_agg(df, secondgroup='unique_episode')

AGENT AND NETWORK
{'network': 'DEEP_CONVOLUTIONAL_NETWORK',
 'sensor_list': ['FRONT_FACING_CAMERA'],
 'simapp_version': '4'}
-------------
HYPERPARAMETERS
{'batch_size': 64,
 'beta_entropy': 0.02,
 'discount_factor': 0.985,
 'e_greedy_value': 0.05,
 'epsilon_steps': 10000,
 'exploration_type': 'categorical',
 'loss_type': 'huber',
 'lr': 5e-05,
 'num_episodes_between_training': 30,
 'num_epochs': 10,
 'stack_size': 1,
 'term_cond_avg_score': 100000.0,
 'term_cond_max_episodes': 100000}
-------------
ACTION SPACE
{'speed': {'high': 4, 'low': 0.8}, 'steering_angle': {'high': 30, 'low': -30}}
new reward not found, using reward as its values


If the code above worked, you will see a list of details printed above: a bit about the agent and the network, a bit about the hyperparameters and some information about the action space. Now let's see what got loaded into the dataframe - the data structure holding your simulation information. the `head()` method prints out a few first lines of the data:

In [19]:
df.head()

,episode,steps,x,y,heading,steering_angle,speed,action,reward,done,...,progress,closest_waypoint,track_len,tstamp,episode_status,pause_duration,iteration,worker,unique_episode,new_reward
0,0,1.0,0.615897,0.462156,131.576337,-5.316137,0.800000,-1,0.000000,False,...,0.421164,0,33.275265,25.484,prepare,0.0,0,0,0,0.000000
1,0,2.0,0.615992,0.462047,131.577320,-1.603432,0.800000,-1,0.421733,False,...,0.420733,0,33.275265,25.545,in_progress,0.0,0,0,0,0.421733
2,0,3.0,0.609900,0.468800,131.557162,-5.381538,0.833191,-1,0.028327,False,...,0.448065,0,33.275265,25.611,in_progress,0.0,0,0,0,0.028327
3,0,4.0,0.597238,0.486175,131.002030,3.242299,0.800000,-1,0.065038,False,...,0.512375,1,33.275265,25.668,in_progress,0.0,0,0,0,0.065038
4,0,5.0,0.576002,0.511877,130.718305,21.506432,0.961543,-1,0.100462,False,...,0.612509,1,33.275265,25.737,in_progress,0.0,0,0,0,0.100462


In [20]:
df.describe()

,episode,steps,x,y,heading,steering_angle,speed,action,reward,progress,closest_waypoint,track_len,tstamp,pause_duration,iteration,worker,unique_episode,new_reward
count,293718.000000,293718.000000,293718.000000,293718.000000,293718.000000,293718.000000,293718.000000,293718.0,293718.000000,293718.000000,293718.000000,2.937180e+05,293718.000000,293718.0,293718.000000,293718.000000,293718.000000,293718.000000
mean,651.288219,71.926096,-1.306616,-0.786046,6.700249,2.749211,1.419405,-1.0,0.154198,21.676546,52.788488,3.327527e+01,11537.664048,0.0,64.654104,1.000667,1954.376964,0.154198
std,380.583196,67.379745,2.142386,2.365004,96.582573,17.023897,0.402353,0.0,0.155770,21.885330,29.851893,2.131632e-14,7308.922825,0.0,38.052994,0.816785,1141.664311,0.155770
min,0.000000,1.000000,-23.288947,-15.010495,-179.986587,-30.000000,0.800000,-1.0,0.000000,-14.575520,0.000000,3.327527e+01,23.258000,0.0,0.000000,0.000000,0.000000,0.000000
25%,322.000000,20.000000,-3.177287,-2.991787,-79.288467,-8.471869,1.110965,-1.0,0.001000,4.314482,28.000000,3.327527e+01,5403.160000,0.0,32.000000,0.000000,967.250000,0.001000
50%,651.000000,49.000000,-1.695540,-1.161569,-8.997960,1.855003,1.465391,-1.0,0.158281,14.325166,56.000000,3.327527e+01,10861.807000,0.0,65.000000,1.000000,1955.000000,0.158281
75%,978.000000,106.000000,-0.135367,1.294665,103.943275,14.040826,1.708632,-1.0,0.238404,32.616777,76.000000,3.327527e+01,16874.936250,0.0,97.000000,2.000000,2932.000000,0.238404
max,1319.000000,340.000000,11.284238,22.132129,179.996777,30.000000,4.000000,-1.0,3.322692,100.000000,111.000000,3.327527e+01,26444.920000,0.0,131.000000,2.000000,3959.000000,3.322692


In [21]:
df['prev_progress'] = df['progress'].shift(1).fillna(0)
df['progress_diff'] = df['progress'] - df['prev_progress']
df[df['steps'] == 1]['prev_progress'] = 0

In [22]:
df['progress_diff'].median()

0.2660426430827708

In [23]:
feature_interests = ['steps', 'reward', 'new_reward']

def calc_discount_factor(r, fr):

    discount_factor = hyperparameters['discount_factor']
    
    current_step = r['steps']
    remaining_steps = fr[(fr['steps'] >= current_step)]

    agg_discount_factors_sum = sum([ discount_factor ** i * r for i ,r  in enumerate(remaining_steps['reward'].values)])
    return agg_discount_factors_sum
    

def get_discounted_reward_for_step(gr):
    gr['discounted_reward'] = gr.apply(lambda r: calc_discount_factor(r[feature_interests], gr[feature_interests]),axis=1)
    return gr

In [24]:
# group = df.groupby('unique_episode')
# group['discounted_reward'] = group.rolling(10)['reward'].sum()

In [25]:
# discount_cal_df = df.set_index('unique_episode')
# discount_cal_df = df.loc[0:20]


# discount_cal_df.groupby('unique_episode').apply(lambda x: print(x['reward']))
df = df.groupby('unique_episode').apply(get_discounted_reward_for_step, )

In [ ]:
[]

## Load waypoints for the track you want to run analysis on

The track waypoint files represent the coordinates of characteristic points of the track - the center line, inside border and outside border. Their main purpose is to visualise the track in images below.

The naming of the tracks is not super consistent. The ones that we already know have been mapped to their official names in the track_meta dictionary.

Some npy files have an 'Eval' suffix. One of the challenges in the past was that the evaluation tracks were different to physical tracks and we have recreated them to enable evaluation. Remeber that evaluation npy files are a community effort to visualise the tracks in the trainings, they aren't 100% accurate.

Tracks Available:

In [ ]:
from deepracer.tracks import TrackIO, Track
from deepracer.tracks.track_utils import track_breakdown, track_meta
tu = TrackIO()

for track in tu.get_tracks():
    print("{} - {}".format(track, track_meta.get(track[:-4], "I don't know")))

Now let's load the track:

In [ ]:
# We will try to guess the track name first, if it 
# fails, we'll use the constant in quotes

try:
    track_name = log.agent_and_network()["world"]
except Exception as e:
    track_name = "2022_reinvent_champ"


track: Track = tu.load_track(track_name)

pu.plot_trackpoints(track)

## Graphs

The original notebook has provided some great ideas on what could be visualised in the graphs. Below examples are a slightly extended version. Let's have a look at what they are presenting and what this may mean to your training.

### Training progress

As you have possibly noticed by now, training episodes are grouped into iterations and this notebook also reflects it. What also marks it are checkpoints in the training. After each iteration a set of ckpt files is generated - they contain outcomes of the training, then a model.pb file is built based on that and the car begins a new iteration. Looking at the data grouped by iterations may lead you to a conclusion, that some earlier checkpoint would be a better start for a new training. While this is limited in the AWS DeepRacer Console, with enough disk space you can keep all the checkpoints along the way and use one of them as a start for new training (or even as a submission to a race).

While the episodes in a given iteration are a mixture of decision process and random guesses, mean results per iteration may show a specific trend. Mean values are accompanied by standard deviation to show the concentration of values around the mean.

#### Rewards per Iteration

You can see these values as lines or dots per episode in the AWS DeepRacer console. When the reward goes up, this suggests that a car is learning and improving with regards to a given reward function. **This does not have to be a good thing.** If your reward function rewards something that harms performance, your car will learn to drive in a way that will make results worse.

At first the rewards just grow if the progress achieved grows. Interesting things may happen slightly later in the training:

* The reward may go flat at some level - it might mean that the car can't get any better. If you think you could still squeeze something better out of it, review the car's progress and consider updating the reward function, the action space, maybe hyperparameters, or perhaps starting over (either from scratch or from some previous checkpoint)
* The reward may become wobbly - here you will see it as a mesh of dots zig-zagging. It can be a gradually growing zig-zag or a roughly stagnated one. This usually means the learning rate hyperparameter is too high and the car started doing actions that oscilate around some local extreme. You can lower the learning rate and hope to step closer to the extreme. Or run away from it if you don't like it
* The reward plunges to near zero and stays roughly flat - I only had that when I messed up the hyperparameters or the reward function. Review recent changes and start training over or consider starting from scratch

The Standard deviation says how close from each other the reward values per episode in a given iteration are. If your model becomes reasonably stable and worst performances become better, at some point the standard deviation may flat out or even decrease. That said, higher speeds usually mean there will be areas on track with higher risk of failure. This may bring the value of standard deviation to a higher value and regardless of whether you like it or not, you need to accept it as a part of fighting for significantly better times.

#### Time per iteration

I'm not sure how useful this graph is. I would worry if it looked very similar to the reward graph - this could suggest that slower laps will be getting higher rewards. But there is a better graph for spotting that below.

#### Progress per Iteration

This graph usually starts low and grows and at some point it will get flatter. The maximum value for progress is 100% so it cannot grow without limits. It usually shows similar initial behaviours to reward and time graphs. I usually look at it when I alter an action in training. In such cases this graph usually dips a bit and then returns or goes higher.

#### Total reward per episode

This graph has been taken from the orignal notebook and can show progress on certain groups of behaviours. It usually forms something like a triangle, sometimes you can see a clear line of progress that shows some new way has been first taught and then perfected.

#### Mean completed lap times per iteration

Once we have a model that completes laps reasonably often, we might want to know how fast the car gets around the track. This graph will show you that. I use it quite often when looking for a model to shave a couple more miliseconds. That said it has to go in pair with the last one:

#### Completion rate per iteration

It represents how big part of all episodes in an iteration is full laps. The value is from range [0, 1] and is a result of deviding amount of full laps in iteration by amount of all episodes in iteration. I say it has to go in pair with the previous one because you not only need a fast lapper, you also want a race completer.

The higher the value, the more stable the model is on a given track.

In [ ]:
simulation_agg = au.simulation_agg(df, secondgroup="unique_episode")
simulation_agg
au.analyze_training_progress(simulation_agg, title='Training progress')

### Stats for all laps

Previous graphs were mainly focused on the state of training with regards to training progress. This however will not give you a lot of information about how well your reward function is doing overall.

In such case `scatter_aggregates` may come handy. It comes with three types of graphs:
* progress/steps/reward depending on the time of an episode - of this I find reward/time and new_reward/time especially useful to see that I am rewarding good behaviours - I expect the reward to time scatter to look roughly triangular
* histograms of time and progress - for all episodes the progress one is usually quite handy to get an idea of model's stability
* progress/time_if_complete/reward to closest waypoint at start - these are really useful during training as they show potentially problematic spots on track. It can turn out that a car gets best reward (and performance) starting at a point that just cannot be reached if the car starts elsewhere, or that there is a section of a track that the car struggles to get past and perhaps it's caused by an aggressive action space or undesirable behaviour prior to that place

Side note: `time_if_complete` is not very accurate and will almost always look better for episodes closer to 100% progress than in case of those 50% and below.

In [ ]:
au.scatter_aggregates(simulation_agg, 'Stats for all laps')

### Stats for complete laps
The graphs here are same as above, but now I am interested in other type of information:
* does the reward scatter show higher rewards for lower completion times? If I give higher reward for a slower lap it might suggest that I am training the car to go slow
* what does the time histogram look like? With enough samples available the histogram takes a normal distribution graph shape. The lower the mean value, the better the chance to complete a fast lap consistently. The longer the tails, the greater the chance of getting lucky in submissions
* is the car completing laps around the place where the race lap starts? Or does it only succeed if it starts in a place different to the racing one?

In [ ]:
complete_ones = simulation_agg[simulation_agg['progress']==100]

if complete_ones.shape[0] > 0:
    au.scatter_aggregates(complete_ones, 'Stats for complete laps')
else:
    print('No complete laps yet.')

### Categories analysis
We're going back to comparing training results based on the training time, but in a different way. Instead of just scattering things in relation to iteration or episode number, this time we're grouping episodes based on a certaing information. For this we use function:
```
analyze_categories(panda, category='quintile', groupcount=5, title=None)
```
The idea is pretty simple - determine a way to cluster the data and provide that as the `category` parameter (alongside the count of groups available). In the default case we take advantage of the aggregated information to which quintile an episode belongs and thus build buckets each containing 20% of episodes which happened around the same time during the training. If your training lasted for five hours, this would show results grouped per each hour.

A side note: if you run the function with `category='start_at'` and `groupcount=20` you will get results based on the waypoint closest to the starting point of an episode. If you need to, you can introduce other types of categories and reuse the function.

The graphs are similar to what we've seen above. I especially like the progress one which shows where the model tends to struggle and whether it's successful laps rate is improving or beginning to decrease. Interestingly, I also had cases where I saw the completion drop on the progress rate only to improve in a later quintile, but with a better time graph.

A second side note: if you run this function for `complete_ones` instead of `simulation_agg`, suddenly the time histogram becomes more interesting as you can see whether completion times improve.

au.scatter_by_groups(simulation_agg, title='Quintiles')

### Last iteration competion rate

In [ ]:
last_iteration = simulation_agg[simulation_agg['iteration'] >= simulation_agg['iteration'].max() - 100]

amount_complete_laps = len(last_iteration[simulation_agg['progress']==100])
total_laps = len(last_iteration)
total_laps

str(round(amount_complete_laps/total_laps * 100)) + '%'

## Data in tables

While a lot can be seen in graphs that cannot be seen in the raw numbers, the numbers let us get into more detail. Below you will find a couple examples. If your model is behaving the way you would like it to, below tables may provide little added value, but if you struggle to improve your car's performance, they may come handy. In such cases I look for examples where high reward is giving to below-expected episode and when good episodes are given low reward.

You can then take the episode number and scatter it below, and also look at reward given per step - this can in turn draw your attention to some rewarding anomalies and help you detect some unexpected outcomes in your reward function.

There is a number of ways to select the data for display:
* `nlargest`/`nsmallest` lets you display information based on a specific value being highest or lowest
* filtering based on a field value, for instance `df[df['episode']==10]` will display only those steps in `df` which belong to episode 10
* `head()` lets you peek into a dataframe

There isn't a right set of tables to display here and the ones below may not suit your needs. Get to know Pandas more and have fun with them. It's almost as addictive as DeepRacer itself.

The examples have a short comment next to them explaining what they are showing.

In [ ]:
# View ten best rewarded episodes in the training
simulation_agg.nlargest(10, 'new_reward')

In [ ]:
# View five fastest complete laps
complete_ones.nsmallest(5, 'time')

In [ ]:
# View the average lap time for completed laps
complete_ones['time'].mean()

In [ ]:
complete_ones['time'].rolling(100).mean().plot()

In [ ]:
# View five best rewarded completed laps
complete_ones.nlargest(5, 'reward')

In [ ]:
# View five best rewarded in completed laps (according to new_reward if you are using it)
complete_ones.nlargest(5, 'new_reward')

In [ ]:
# View five most progressed episodes
simulation_agg.nlargest(5, 'progress')

In [ ]:
# View information for a couple first episodes
simulation_agg.head()

In [ ]:
# Set maximum quantity of rows to view for a dataframe display - without that
# the view below will just hide some of the steps
pd.set_option('display.max_rows', 500)

# View all steps data for episode 10
df[df['episode']==10].head()

In [ ]:
most_progressed_episode =simulation_agg[simulation_agg['progress']==simulation_agg['progress'].max()]

fastest_episode =  int(most_progressed_episode['unique_episode'].iloc[0])
fastest_iteration = int(most_progressed_episode['iteration'].iloc[0])
fastest_iteration,fastest_episode

In [ ]:
if len(complete_ones) != 0:
    fastest_episode = int(complete_ones.nsmallest(1, 'time').iloc[0]['unique_episode'])
    fastest_iteration = int(complete_ones.nsmallest(1, 'time').iloc[0]['iteration'])

In [ ]:
# df[df['unique_episode'] == 5190][['steps','progress', 'prev_progress', 'progress_diff', 'reward', 'discounted_reward']]

## Analyze the reward distribution for your reward function

In [ ]:
# This shows a histogram of actions per closest waypoint for episode 889.
# Will let you spot potentially problematic places in reward granting.
# In this example reward function is clearly `return 1`. It may be worrying
# if your reward function has some logic in it.
# If you have a final step reward that makes the rest of this histogram
# unreadable, you can filter the last step out by using
# `episode[:-1].plot.bar` instead of `episode.plot.bar`
# episode = df[df['episode']==889]
episode = df[df['unique_episode']==fastest_episode]


if episode.empty:
    print("You probably don't have episode with this number, try a lower one.")
else:
    episode.plot.bar(x='closest_waypoint', y='reward')

### Path taken for top reward iterations

NOTE: at some point in the past in a single episode the car could go around multiple laps, the episode was terminated when car completed 1000 steps. Currently one episode has at most one lap. This explains why you can see multiple laps in an episode plotted below.

Being able to plot the car's route in an episode can help you detect certain patterns in its behaviours and either promote them more or train away from them. While being able to watch the car go in the training gives some information, being able to reproduce it after the training is much more practical.

Graphs below give you a chance to look deeper into your car's behaviour on track.

We start with plot_selected_laps. The general idea of this block is as follows:
* Select laps(episodes) that have the properties that you care about, for instance, fastest, most progressed, failing in a certain section of the track or not failing in there,
* Provide the list of them in a dataframe into the plot_selected_laps, together with the whole training dataframe and the track info,
* You've got the laps to analyse.

In [ ]:
# Some examples:
# highest reward for complete laps:
# episodes_to_plot = complete_ones.nlargest(3,'reward')

# highest progress from all episodes:
# episodes_to_plot = simulation_agg.nlargest(3,'progress')

# fastest laps from all episodes:
episodes_to_plot = complete_ones.nsmallest(3,'time')

pu.plot_selected_laps(episodes_to_plot, df, track, section_to_plot="unique_episode")

In [ ]:
import pandas as pd

start_index = 100000

speed_mean_wp = df[start_index:].groupby('closest_waypoint')['speed'].mean()
x_mean_wp = df[start_index:].groupby('closest_waypoint')['x'].mean()
y_mean_wp = df[start_index:].groupby('closest_waypoint')['y'].mean()


average_lap_df = pd.DataFrame({'speed': speed_mean_wp, 'x': x_mean_wp, 'y': y_mean_wp}).reset_index()
average_lap_df['unique_episode'] = 1

pu.plot_selected_laps(average_lap_df, average_lap_df, track, section_to_plot="unique_episode")

### Plot a heatmap of rewards for current training. 
The brighter the colour, the higher the reward granted in given coordinates.
If instead of a similar view as in the example below you get a dark image with hardly any 
dots, it might be that your rewards are highly disproportionate and possibly sparse.

Disproportion means you may have one reward of 10.000 and the rest in range 0.01-1.
In such cases the vast majority of dots will simply be very dark and the only bright dot
might be in a place difficult to spot. I recommend you go back to the tables and show highest
and average rewards per step to confirm if this is the case. Such disproportions may
not affect your traning very negatively, but they will make the data less readable in this notebook.

Sparse data means that the car gets a high reward for the best behaviour and very low reward
for anything else, and worse even, reward is pretty much discrete (return 10 for narrow perfect,
else return 0.1). The car relies on reward varying between behaviours to find gradients that can
lead to improvement. If that is missing, the model will struggle to improve.

In [ ]:
#If you'd like some other colour criterion, you can add
#a value_field parameter and specify a different column
pu.plot_track(df, track)
# pu.plot_track(df[:10000], track)
# pu.plot_track(df[1000:], track)
# pu.plot_track(df[df['unique_episode'] == complete_ones['unique_episode']], track)
# pu.plot_track(df, track)
# pu.plot_track(df[(df['x'] > 0) & (df['y'] > 0) & (df['y'] < df['y'].max()) & (df['y'] > df['y'].min())& (df['x'] < df['x'].max()) & (df['x'] > df['x'].min())] , track)
# pu.plot_track(df[(df['y'] < df['y'].max()) & (df['y'] > df['y'].min())& (df['x'] < df['x'].max()) & (df['x'] > df['x'].min())] , track)

### Plot including discount factor

In [ ]:
# pu.plot_track(df[df['discounted_reward'] <  df['discounted_reward'].max() * 0.9], track, 'discounted_reward')
# pu.plot_track(df, track, 'discounted_reward')
pu.plot_track(df.tail(int(len(df) * 0.9)), track, 'discounted_reward')

### Plot a particular iteration
This is same as the heatmap above, but just for a single iteration.

In [ ]:
#If you'd like some other colour criterion, you can add
#a value_field parameter and specify a different column
# iteration_id = 304
iteration_id=fastest_iteration
pu.plot_track(df[df['iteration'] == iteration_id], track)

### Path taken in a particular episode

In [ ]:
# episode_id = fastest_episode
# episode_id = 142 #856
episode_id = 856

pu.plot_selected_laps(df[df['unique_episode']==episode_id], df, track,single_plot=False)

### Path taken in a particular iteration

In [ ]:
iteration_id = int(fastest_iteration)

pu.plot_selected_laps([iteration_id], df, track, section_to_plot = 'iteration')

# Action breakdown per iteration and historgram for action distribution for each of the turns - reinvent track

This plot is useful to understand the actions that the model takes for any given iteration. Unfortunately at this time it is not fit for purpose as it assumes six actions in the action space and has other issues. It will require some work to get it to done but the information it returns will be very valuable.

This is a bit of an attempt to abstract away from the brilliant function in the original notebook towards a more general graph that we could use. It should be treated as a work in progress. The track_breakdown could be used as a starting point for a general track information object to handle all the customisations needed in methods of this notebook.

A breakdown track data needs to be available for it. If you cannot find it for the desired track, MAKEIT.

Currently supported tracks:

In [ ]:
track_breakdown.keys()

You can replace episode_ids with iteration_ids and make a breakdown for a whole iteration.

**Note: does not work for continuous action space (yet).** 

In [ ]:
abu.action_breakdown(df, track, track_breakdown=track_breakdown.get(track_name), episode_ids=[fastest_episode])

Look at the grid world

In [ ]:
# df[df['episode']== fastest_episode]
episode = df[df['unique_episode']==fastest_episode]

In [ ]:
pu.plot_grid_world(episode, track)

# Fault Finding

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
complete_laps = simulation_agg[simulation_agg['progress']==100]

In [ ]:
without_possible_off_tracks =  complete_laps[complete_laps['reward'] > 95]
and_possible_going_crazy = without_possible_off_tracks[without_possible_off_tracks['reward'] < 105]

In [ ]:
# plt.title.set_text('Reward vs Time for completed laps')  # type: ignore
# plt.set_xlabel('Time')
# plt.set_ylabel('Reward')
plt.scatter(and_possible_going_crazy['time'], and_possible_going_crazy['reward'], linewidth=2)

In [ ]:
negative_reward = simulation_agg[simulation_agg['progress']<0]

In [ ]:
this = complete_laps[complete_laps['reward'] > 105]

In [ ]:
df[df['unique_episode']==episode_id]

In [ ]:
faulty_mask = df[(df['unique_episode']==episode_id) & (df['reward'] > 1)]
faulty_mask = df[df['reward'] > 1]

faulty_mask

In [ ]:

pu.plot_selected_laps(faulty_mask, faulty_mask, track,single_plot=False)